In [36]:
from __future__ import print_function
import cv2 as cv
import os
import numpy as np

def process_background_subtraction(image_folder):
    background_subtractors = [
        cv.createBackgroundSubtractorMOG2(history=1000),
        # cv.createBackgroundSubtractorKNN()
    ]
    
    fourcc = cv.VideoWriter_fourcc(*'mp4v') 
    fps = 30
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]
    first_frame = cv.imread(os.path.join(image_folder, image_files[0]))
    if first_frame is not None:
        frame_size = (first_frame.shape[1], first_frame.shape[0])
    else:
        raise ValueError("No valid images found in the specified directory.")

    for backSub in background_subtractors:
        # set params
        backSub.setDetectShadows(False)
        backSub.setBackgroundRatio(0.99999)
        
        sub_dir = f'./output/output_{backSub.__class__.__name__}'
        os.makedirs(sub_dir, exist_ok=True)
        output_file = os.path.join(sub_dir, f'output_{os.path.basename(image_folder)}.mp4')
        out = cv.VideoWriter(output_file, fourcc, fps, frame_size)

        for image_file in image_files:
            image_path = os.path.join(image_folder, image_file)
            frame = cv.imread(image_path)

            if frame is None:
                print('Unable to open: ' + image_path)
                continue

            fgMask = backSub.apply(frame)

            # Remove noise from the foreground mask
            noise_removal_kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
            cleanedMask = cv.morphologyEx(fgMask, cv.MORPH_OPEN, noise_removal_kernel)

            # Apply closing operation to fill small holes
            closedMask = cv.morphologyEx(cleanedMask, cv.MORPH_CLOSE, noise_removal_kernel)

            # Detect circles in the closed mask
            circles = cv.HoughCircles(closedMask, cv.HOUGH_GRADIENT, dp=1, minDist=20,
                                       param1=50, param2=30, minRadius=1, maxRadius=2)

            # If circles are detected, draw them on the frame
            if circles is not None:
                circles = np.uint16(np.around(circles))
                for i in circles[0, :]:
                    # Draw the outer circle
                    cv.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
                    # Draw the center of the circle
                    cv.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 3)

            # Write the original frame and the closed mask to the video
            out.write(frame)
            out.write(closedMask)

            cv.rectangle(frame, (10, 2), (100, 20), (255, 255, 255), -1)
            cv.putText(frame, str(image_file), (15, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

            cv.imshow('Frame', frame)
            cv.imshow(f'Closed FG Mask', closedMask)

            keyboard = cv.waitKey(30)
            if keyboard == ord('q') or keyboard == 27:
                break

        out.release()
        cv.destroyAllWindows()

clip_directories = ['./dataset/train/images/clip_one', 
                    './dataset/train/images/clip_two', 
                    './dataset/train/images/clip_three', 
                    './dataset/train/images/clip_four']

for clip_dir in clip_directories:
    process_background_subtraction(clip_dir)

In [1]:
from __future__ import print_function
import cv2 as cv
import os

def process_background_subtraction(image_folder):
    background_subtractors = [
        cv.createBackgroundSubtractorMOG2(),
        cv.createBackgroundSubtractorKNN(),
        cv.bgsegm.createBackgroundSubtractorMOG(),
        cv.bgsegm.createBackgroundSubtractorGMG()
    ]
    
    
# The GMG (Geodesic Object Tracking) background subtractor may only show results near the end of the video due to its reliance on a learning process that requires a sufficient number of frames to adapt to the scene. If the initial frames do not contain enough information for it to learn the background effectively, it may take longer to produce accurate foreground masks.
    
    fourcc = cv.VideoWriter_fourcc(*'mp4v') 
    fps = 30
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]
    first_frame = cv.imread(os.path.join(image_folder, image_files[0]))
    if first_frame is not None:
        frame_size = (first_frame.shape[1] * 2, first_frame.shape[0] * 2)
    else:
        raise ValueError("No valid images found in the specified directory.")

    sub_dir = './output/output_combined'
    os.makedirs(sub_dir, exist_ok=True)
    output_file = os.path.join(sub_dir, f'output_{os.path.basename(image_folder)}.mp4')
    out = cv.VideoWriter(output_file, fourcc, fps, frame_size)

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        frame = cv.imread(image_path)

        if frame is None:
            print('Unable to open: ' + image_path)
            continue

        fgMasks = [bg_subtractor.apply(frame) for bg_subtractor in background_subtractors]
        fgMasks_colored = [cv.cvtColor(fgMask, cv.COLOR_GRAY2BGR) for fgMask in fgMasks]

        # Arrange the frames in a 2x2 grid
        combined_frame = cv.vconcat([
            cv.hconcat(fgMasks_colored[:2]),
            cv.hconcat(fgMasks_colored[2:])
        ])

        for i, bg_subtractor in enumerate(background_subtractors):
            cv.putText(combined_frame, bg_subtractor.__class__.__name__, (10 + (i % 2) * (frame_size[0] // 2), 30 + (i // 2) * (frame_size[1] // 2)), cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        out.write(combined_frame)

        cv.imshow('Combined Frame', combined_frame)

        keyboard = cv.waitKey(30)
        if keyboard == ord('q') or keyboard == 27:
            break

    out.release()
    cv.destroyAllWindows()

clip_directories = ['./dataset/train/images/clip_one', 
                    './dataset/train/images/clip_two', 
                    './dataset/train/images/clip_three', 
                    './dataset/train/images/clip_four']

for clip_dir in clip_directories:
    process_background_subtraction(clip_dir)

In [32]:
# %pip install opencv-python opencv-contrib-python --user

  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2 as cv

input_file = './output/output_combined/output_clip_two.mp4'
cap = cv.VideoCapture(input_file)

original_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

new_width = original_width // 2
new_height = original_height // 2

output_file = './output/output_combined/output_clip_two_downsized.mp4'
fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter(output_file, fourcc, 30.0, (new_width, new_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    downsized_frame = cv.resize(frame, (new_width, new_height))
    
    out.write(downsized_frame)

cap.release()
out.release()
cv.destroyAllWindows()
